In [1]:
run model_deployment.py

Getting data...
Data pulled successfully, cleanning data...
Training models...
silhouette score using new features: 0.7517334427221185
silhouette score using all features: 0.7567877361460673


/libraries/driver_fraud_anomaly_env/lib/python3.7/site-packages/xlsxwriter/workbook.py:369: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [1]:
from model_deployment import get_config, get_data, clean_data, train_k_means, analyze_driver_groups, create_excel_file, create_log, send_email  

In [3]:
import os
import json
import pytz
from datetime import datetime

import pandas as pd
import numpy as np

# Bigquery
from google.cloud import bigquery

try:
    from mlutils import connector # Element environment connectors
    APP_PATH = '/home/jupyter/Driver-Fraud-Anomaly-Predictive-Model/'
except ImportError:
    APP_PATH = ""
    
# Machine Learning
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

#Email Utils
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import smtplib
import io

# pull today's date
EMAIL_DATE = datetime.now(pytz.timezone('America/Los_Angeles')).strftime("%Y-%m-%d")

In [4]:
config = get_config(APP_PATH, "config.json")
is_success, df, log_body = get_data(config)

C:\Users\y0q00wv\Anaconda3\lib\site-packages\google\auth\_default.py:70: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [5]:
is_success

True

In [6]:
if is_success:
    clean_df = clean_data(df, config)

In [7]:
# train using both new_features and all features
trained_df_new, kmeans_new, silhouette_score_value_new = train_k_means(clean_df, config, "new_features")
trained_df_all, kmeans_all, silhouette_score_value_all = train_k_means(clean_df, config, "all_features")
print(f"silhouette score using new features: {silhouette_score_value_new}")
print(f"silhouette score using all features: {silhouette_score_value_all}")

C:\Users\y0q00wv\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\y0q00wv\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


silhouette score using new features: 0.7385904917101864
silhouette score using all features: 0.734066196315693


In [8]:
# output result
result_df_new, detail_df_new = analyze_driver_groups(trained_df_new, config, "new_features")
result_df_all, detail_df_all = analyze_driver_groups(trained_df_all, config, "all_features")

# create excel attachment
X = config['clusters_to_share']

df_dict = {
    'Summary_New_Features': result_df_new,
    f'Dtl_Least{X}Clstrs_NewFtrs': detail_df_new,
    'Summary_All_Features': result_df_all,
    f'Dtl_Least{X}Clstrs_AllFtrs': detail_df_all
}
attachment = create_excel_file(df_dict, config)
msg = create_log(log_body, config, is_success, attachment=attachment)
send_email(msg)

C:\Users\y0q00wv\Documents\GitHub\Driver-Fraud-Anomoly-Predictive-Model\model_deployment.py:172: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  with pd.ExcelWriter(buffer) as writer:


AttributeError: 'OpenpyxlWriter' object has no attribute 'save'

In [ ]:
# test on fail message
fail_log_body = "TEST on failed message"
success = False
msg = create_log(fail_log_body, config, success)
send_email(msg)

Share some driver samples to stakeholder

In [20]:
result_df_new.to_clipboard(index=False)

In [21]:
result_df_all.to_clipboard(index=False)

In [23]:
trained_df_all[trained_df_all.driver_group_all_features == 2].DEACTIVATION_RSN_NM.value_counts()

DEACTIVATION_RSN_NM
Platform Manipulation    67
Critical - Other         11
Critical                 11
Behavior                  5
Order Theft               4
Name: count, dtype: int64

In [26]:
sampled_df_new = trained_df_new.groupby('driver_group_new_features').apply(lambda x: x.sample(n=50))

In [33]:
sampled_df_new.reset_index(drop=True, inplace=True)
sampled_df_new.sort_values('driver_group_new_features', inplace=True)

In [35]:
sampled_df_new.to_clipboard(index=False)

In [38]:
trained_df_new.data_end_dt_pt

0         2024-11-13
1         2024-11-13
2         2024-11-13
3         2024-11-13
4         2024-11-13
             ...    
664343    2024-11-13
664344    2024-11-13
664345    2024-11-13
664346    2024-11-13
664347    2024-11-13
Name: data_end_dt_pt, Length: 664348, dtype: object